# Project 5 Person of Interest
## A detection and tracking system for public cameras

## Part II TensorFlow object detection system and combination with tracker

## After training two deep learning models, combine tracker with object detection, to find the count of pedestrians in the video and their speed. 

Person of Interest: OpenCV Tracking API | TensorFlow | Deep learning | AWS
- Enable the camera watch for public security at New York Time Square.
- Developing a detection and tracking system for one of the public cameras.
- Setting notification of abnormal counts of people and their moving speed.

In [1]:
import cv2
import numpy as np
import gc
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from utils import label_map_util
from utils import visualization_utils as vis_util

%matplotlib inline

In [8]:
# ! ls 

## find velocity of pedestrian

In [ ]:
# find velocity of pedestrian
import numpy as np
import cv2
import sys

cv2.namedWindow("tracking")
camera = cv2.VideoCapture('../../../nyc17_ds12/projects/05-kojak/videos/less/media_w922939858_9.ts')
ok, image=camera.read()
if not ok:
    print('Failed to read video')
    exit()

#bounding box, region of interest !!! #bbox=(x,y,w,h)  
bbox = cv2.selectROI("tracking", image)
print(bbox)

x0=int(bbox[0])
y0=int(bbox[1])
count=0
ave_v=[]
ave_vx=[]

tracker_mil = cv2.TrackerMIL_create()
init_once = False

while camera.isOpened():
    ok, image=camera.read()
    if not ok:
        print ('no image to read')
        break

    if not init_once:
        ok = tracker_mil.init(image,bbox)
        if not ok:
            print ('tracker %i init error' % i)
            break
        init_once = True

    ok, newbox = tracker_mil.update(image)

    if ok:
        p1=(int(newbox[0]),int(newbox[1]))
        p2=(int(newbox[0]+newbox[2]),int(newbox[1]+newbox[3]))
        cv2.rectangle(image,p1,p2,(0,200,0),2)

        x=int(newbox[0])
        y=int(newbox[1])
        
        if count == 10:
            update_v(sum(ave_v)/10)
            update_v(sum(ave_vx)/10)
            #reset velocity list and count
            count=1
            ave_v=[np.sqrt((x-x0)**2 + (y-y0)**2)]
            ave_vx=[x-x0]
        else:
            ave_v.append(np.sqrt((x-x0)**2 + (y-y0)**2))
            ave_vx.append(x-x0)            
            count+=1

        x0 = x
        y0 = y

    cv2.imshow("tracking", image)
    k = cv2.waitKey(30) & 0xff
    if k == 27 : break # esc pressed
        
camera.release()
cv2.destroyAllWindows()
for i in range(1,5):
    cv2.waitKey(1)

In [2]:
import os
import numpy as np
import tensorflow as tf

# Path to frozen detection graph. This is the actual model that is used for the object detection.
MODEL_NAME = 'pedestrian_graph_aws5'
# MODEL_NAME = 'pedestrian_graph_aws_rcnn'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training', 'pedestrian_label_lessclass.pbtxt')

NUM_CLASSES = 3

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [3]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [4]:
def detect_objects(image_np,sess,detection_graph):
    #get image width and height from shape
    w=image_np.shape[1]
    h=image_np.shape[0]
        
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    
    # Each box represents a part of the image where a particular object was detected.
    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    
    # Actual detection.
    (boxes, scores, classes, num_detections) = sess.run(  
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
    squeezed_boxes = np.squeeze(boxes)
    
    i=0
    while scores[0][i]>=0.5:
        i+=1
    
    bboxes=[]
    for j in range(i):
        y = int(h*boxes[0][j][0])
        x = int(w*boxes[0][j][1])
        hy = int(h*(boxes[0][j][2]-boxes[0][j][0]))
        wx = int(w*(boxes[0][j][3]-boxes[0][j][1]))
        bboxes.append((x,y,wx,hy))
    return bboxes, classes[0][:i], scores[0][:i]

In [10]:
import cv2

def update_v(v):
    print(v) 
    return

def reset_tracker(image_np,sess,detection_graph):
    bboxes, classes, scores = detect_objects(image_np,sess,detection_graph)
    # set/reset multiple tracker
    tracker = cv2.MultiTracker_create()
    for i in range(len(bboxes)):
        tracker.add(cv2.TrackerMIL_create(), image_np, bboxes[i])
    
    return tracker, classes, scores

def draw_object_boxes(boxes,image_np):
    for i in range(len(boxes)):
        p1 = (int(boxes[i][0]), int(boxes[i][1]))
        p2 = (int(boxes[i][0] + boxes[i][2]), int(boxes[i][1] + boxes[i][3]))
        cv2.rectangle(image_np, p1, p2, (0,200,0),2)

    return image_np

def cal_avg_speed_list(speed_lists):
    if len(speed_lists) == 0:
        return []
    else:
        return [sum(l) / len(l) for l in speed_lists]

def reset_positions(boxes):
    return [(int(box[0]), int(box[1])) for box in boxes]

def reset_speed_lists(dim):
    return [[] for i in range(dim)]

def update_speed_lists(boxes, x_speed_lists, xy_speed_lists,positions):
    n = len(x_speed_lists)
    if n != len(boxes):
        print("Error: speed list length and boxes number not match")
        exit()
    elif n != len(positions):
        print("Error: speed list length and positions number not match")
        exit()
    else:
        for i in range(n):
            x0 = positions[i][1]
            x1 = int(boxes[i][1])
            y0 = positions[i][0]
            y1 = int(boxes[i][0])
            xy_speed_lists[i].append(np.sqrt((x1-x0)**2 + (y1-y0)**2))
            x_speed_lists[i].append(x1-x0)
            positions[i] = [y1,x1]
            
    return (x_speed_lists, xy_speed_lists, positions)
    
#
cv2.namedWindow("tracking")
cap = cv2.VideoCapture('../../../nyc17_ds12/projects/05-kojak/videos/less/n11.mov')
#n14, for ssd -2000 model
#run0 for ssd -10000 model

# Initialize variables
counter = 0
# i=0
x_speed_lists = []
xy_speed_lists = []

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        while(cap.isOpened()):
            # read frame
            ret,image_np = cap.read()
            if not ret:
                break

            # detecting every 10 frames
            if counter%10 == 0:
                # set/reset multiple tracker & counter
                tracker, classes, scores = reset_tracker(image_np, sess, detection_graph)
                counter = 0
            
            # tracking: update object boxes
            ok, boxes = tracker.update(image_np)

            # tracking: draw object boxes
            image_np = draw_object_boxes(boxes, image_np)
            
            # tracking: update average speed list
            if counter == 0:
                x_avg_speed_list = cal_avg_speed_list(x_speed_lists)
                xy_avg_speed_list = cal_avg_speed_list(xy_speed_lists)
                
                positions = reset_positions(boxes)
                x_speed_lists = reset_speed_lists(len(boxes))
                xy_speed_lists = reset_speed_lists(len(boxes))
            else:
                x_speed_lists, xy_speed_lists, positions = update_speed_lists(boxes, x_speed_lists, xy_speed_lists, positions)
            
#             print(len(boxes),'\n')
#             print(boxes)
#             print(x_avg_speed_list,xy_avg_speed_list)
#             break
            
#             vis_util.visualize_boxes_and_labels_on_image_array(
#                   image_np,
#                   np.squeeze(boxes),
#                   np.squeeze(classes).astype(np.int32),
#                   np.squeeze(scores),
#                   {},
#                   use_normalized_coordinates=True,
#                   line_thickness=8)
            
            # increment counter
            counter += 1
#             i += 1
#             cv2.putText(img, 'Tian',(x,y),font,4,(100,100,0),4,cv2.LINE_AA)

            # show marked frame
            cv2.imshow('tracking', image_np)
#             cv2.imwrite( "../../../nyc17_ds12/projects/05-kojak/images/flask_display/n14_101136_%d.jpg" %i, image_np );
            if cv2.waitKey(25) & 0xff == 27:
                break
        
cap.release()
cv2.destroyAllWindows()
for i in range(1,5):
    cv2.waitKey(1)